# Нейроэволюционный алгоритм

## Загружаем и подготавливаем данные для работы

### Строки, не несущие информацию были удалены

In [1]:
import datatable as dt
data = dt.fread('cancer1.dt')
del data[0:7,:]

## Преробразуем данные в формат pandas.DataFrame 

In [2]:
import pandas as pd
df = pd.DataFrame(data.to_pandas())
df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
0,0.2,0.1,0.1,0.1,0.2,0.1,0.2,0.1,0.1,True,False
1,0.2,0.1,0.1,0.1,0.2,0.1,0.3,0.1,0.1,True,False
2,0.5,0.1,0.1,0.1,0.2,0.1,0.2,0.1,0.1,True,False
3,0.5,0.4,0.6,0.8,0.4,0.1,0.8,1.0,0.1,False,True
4,0.5,0.3,0.3,0.1,0.2,0.1,0.2,0.1,0.1,True,False
...,...,...,...,...,...,...,...,...,...,...,...
694,0.5,0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1,True,False
695,0.7,0.5,0.6,1.0,0.5,1.0,0.7,0.9,0.4,False,True
696,0.6,1.0,1.0,1.0,0.8,1.0,0.7,1.0,0.7,False,True
697,0.5,0.7,1.0,1.0,0.5,1.0,1.0,1.0,0.1,False,True


## Разделяем данные на входные и требуемые выходные значения

In [3]:
outputs = df[['C9','C10']] \
                    .iloc[:,0]\
                    .astype('int64')

outputs

0      1
1      1
2      1
3      0
4      1
      ..
694    1
695    0
696    0
697    0
698    1
Name: C9, Length: 699, dtype: int64

In [4]:
inputs = df.iloc[:, 0:9].copy()
inputs

,C0,C1,C2,C3,C4,C5,C6,C7,C8
0,0.2,0.1,0.1,0.1,0.2,0.1,0.2,0.1,0.1
1,0.2,0.1,0.1,0.1,0.2,0.1,0.3,0.1,0.1
2,0.5,0.1,0.1,0.1,0.2,0.1,0.2,0.1,0.1
3,0.5,0.4,0.6,0.8,0.4,0.1,0.8,1.0,0.1
4,0.5,0.3,0.3,0.1,0.2,0.1,0.2,0.1,0.1
...,...,...,...,...,...,...,...,...,...
694,0.5,0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
695,0.7,0.5,0.6,1.0,0.5,1.0,0.7,0.9,0.4
696,0.6,1.0,1.0,1.0,0.8,1.0,0.7,1.0,0.7
697,0.5,0.7,1.0,1.0,0.5,1.0,1.0,1.0,0.1


## Разбиваем выборку на тренировочную и тестовую

In [5]:
from sklearn.model_selection import train_test_split

inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, train_size = 0.8, random_state = 3)